In [1]:
## Importing libraries 
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [2]:
train_speakers = ['2001', '2002', '2003', '2005', '2006', '2007', '2008', '2009', '2010']
val_speakers = ['2011']
train_manifest_path = '/home/dgxuser/Desktop/kailin/audio_files_location/train_manifest2.json'
val_manifest_path = '/home/dgxuser/Desktop/kailin/audio_files_location/val_manifest2.json'
audio_dir = '/home/dgxuser/Desktop/kailin/audio_files_location/'
script_path = '/home/dgxuser/Desktop/kailin/TTS_location_script.txt'

## Function to build a manifest
def build_manifest(manifest_path, speakers):
  count = 0
  with open(manifest_path, 'w') as fout: 
    for i in speakers: 
      audio_path = audio_dir + 'SPEAKER' + i
      n = 0 # change back later 
      with open(script_path, encoding = 'utf-8') as script:
        while os.path.exists(audio_path + '/' + str(n) + '.wav'):
            line = script.readline().strip()
            words = line.split()
            edited_words = []
            for word in words:
              if (word[-1] != '>') and (word != '**'):
                edited_words.append(word.lower())
            transcript_line = ""
            for word in edited_words: 
              transcript_line = transcript_line + word + " "
            transcript_line = transcript_line[:-1]
            #getting audio file
            audio = audio_path + '/' + str(n) + '.wav'
            duration = librosa.core.get_duration(filename=audio)
            # Write the metadata to the manifest
            metadata = {
                "audio_filepath": audio,
                "text": transcript_line,
                "duration": duration,
                "text_no_preprocessing": transcript_line, 
                "text_normalized": transcript_line
            }
            json.dump(metadata, fout)
            fout.write('\n')
            n = n+1

build_manifest(train_manifest_path, train_speakers) 
build_manifest(val_manifest_path, val_speakers)

In [3]:
## Importing libraries 
import nemo
import nemo.collections.asr as nemo_asr
import jiwer
from jiwer import wer
from jiwer import cer
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile
from omegaconf import OmegaConf, open_dict
from nemo.collections.asr.models.ctc_bpe_models import EncDecCTCModelBPE
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager

[NeMo W 2022-07-04 15:50:27 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.


In [4]:
# Initialise quartznet model 
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5")

[NeMo I 2022-07-04 15:50:38 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2022-07-04 15:50:38 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2022-07-04 15:50:38 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-04 15:50:39 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2022-07-04 15:50:39 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2022-07-04 15:50:39 features:200] PADDING: 16
[NeMo I 2022-07-04 15:50:42 save_restore_connector:243] Model EncDecCTCModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.


In [5]:
## Config Information 
try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel_yaml import YAML
config_path = './configs/config.yaml'

yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
print(params)

{'name': 'QuartzNet15x5', 'sample_rate': 16000, 'repeat': 1, 'dropout': 0.0, 'separable': True, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'model': {'train_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'trim_silence': True, 'max_duration': 16.7, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': None, 'shuffle_n': 2048, 'bucketing_strategy': 'synced_randomized', 'bucketing_batch_size': None}, 'validation_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'shuffle': False}, 'preprocessor': {'_target_': 'nemo.collections

In [16]:
train_manifest_path = '/home/dgxuser/Desktop/kailin/audio_files_location/train_manifest2.json'
val_manifest_path = '/home/dgxuser/Desktop/kailin/audio_files_location/val_manifest2.json'

# add in manifest paths 
params['model']['train_ds']['manifest_filepath'] = train_manifest_path
params['model']['validation_ds']['manifest_filepath'] = val_manifest_path

# Point to the new validation data for fine-tuning
quartznet.setup_validation_data(val_data_config=params['model']['validation_ds'])

# Point to the data we'll use for fine-tuning as the training set
quartznet.setup_training_data(train_data_config=params['model']['train_ds'])

# Create a PyTorch Lightning trainer and call `fit` again.
trainer = pl.Trainer(max_epochs=100, accelerator='gpu', devices=1)
trainer.fit(quartznet)

[NeMo I 2022-07-04 16:02:39 collections:193] Dataset loaded with 8 files totalling 0.00 hours
[NeMo I 2022-07-04 16:02:39 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2022-07-04 16:02:39 collections:193] Dataset loaded with 72 files totalling 0.04 hours
[NeMo I 2022-07-04 16:02:39 collections:194] 0 files were filtered totalling 0.00 hours


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
[NeMo W 2022-07-04 16:02:40 modelPT:484] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2022-07-04 16:02:40 modelPT:585] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.01
        weight_decay: 0.001
    )


[NeMo W 2022-07-04 16:02:40 lr_scheduler:816] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 29.7 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
75.698    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [17]:
manifest_path = '/home/dgxuser/Desktop/kailin/location_data (updated)/train_manifest.json'
data_dir = '/home/dgxuser/Desktop/kailin/location_data (updated)/audio_files_train/'
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        path = data_dir + data["audio_filepath"][-14:]
        files = [path]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
print(transcript)

['bishan bishan', 'boon lay boon lay', 'bukit batok bukit batok', 'bukit gombak bukit gombak', 'jurong jurong', 'sembawang sembawang', 'tiong bahru tiong bahru', 'tuas tuas', 'yishun yishun', 'bishan bishan', 'boon lay boon lay', 'bukit batok bukit batok', 'bukit gombak bukit gombak', 'jurong jurong', 'sembawang sembawang', 'tiong bahru tiong bahru', 'tuas tuas', 'yishun yishun', 'boon lay boon lay', 'bishan bishan', 'jurong jurong', 'bishan bishan', 'boon lay boon lay', 'bukit batok bukit batok', 'bukit gombak bukit gombak', 'jurong jurong', 'sembawang sembawang', 'tiong bahru tiong bahru', 'tuas tuas', 'yishun yishun', 'bishan bishan', 'boon lay boon lay', 'bukit batok bukit batok', 'bukit gombak bukit gombak', 'jurong jurong', 'sembawang sembawang', 'tiong bahru tiong bahru', 'tuas tuas', 'bishan bishan', 'boon lay boon lay', 'bukit batok bukit batok', 'bukit gombak bukit gombak', 'jurong jurong', 'sembawang sembawang', 'tiong bahru tiong bahru', 'tuas tuas', 'yishun yishun', 'bisha

In [19]:
print(predicted)

['bishun yishun', 'bo boon laybo boon lay', 'bukit batok bukit batok', 'bbukit gombak bukit gombak', 'jurongjurong', 'sembawangsembawang ', 'jurong bahruturong jaru', 'ju yishunbuon yijum', 'yishun yishun', 'bishun bishn', 'boon lay boon lay', 'kbkitbukit batok bukit batok bk', 'butbukit gombak bukit gombak', 'jurong jurong', 'sembawang sembawang', 'jujung jurong', 'yishun yishun', 'bishun yishun', 'tiong bawan toong ju', 'bolay boshan', 'jurong jurong', 'yishun su yisu', 'boon lay boon lay', 'juong go it batok', 'juijuongtea ba', 'tiong bawn basit gombak', 'jurong gom sembawa ju', 'tiong bahru tu tiong bahru', 'juogjujurong yisu', 'junjishunjurong', 'bishan bishan', 'sembawong boon lay', 'bukit batok bukit batok', 'bukit gombak to bukit gombak', 'jurong jurong', 'sembawang sembawang', 'buong ba sembawang', 'uyishun gomba', 'bishan bishan', 'tong bahrug jong bahru', 'bukit batok bukit batok', 'bukit ombak bukit gombak', 'jurong sembawang', 'sembawang sembawang', 'jurong jombawong', 'yi

In [20]:
error = wer(transcript, predicted)
print('Word Error Rate: ' + str(error))

Word Error Rate: 0.5643564356435643
